In [3]:
#Le plan : CMA MAE -> 3/9 points
#reinforce en loadant le policy network 
a=9709249010.93793
b=2610906993.0028586
from train import *
pi = ProjectAgent()
pi.load()


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
from torch.distributions import Categorical
class reinforce_agent:
    def __init__(self, config, policy_network):
        self.device = "cuda" if next(policy_network.parameters()).is_cuda else "cpu"
        self.scalar_dtype = next(policy_network.parameters()).dtype
        self.policy = policy_network
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.99
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = torch.optim.Adam(list(self.policy.parameters()),lr=lr)

    def sample_action_and_log_prob(self, x):
        logits = self.policy(torch.as_tensor(x).to(torch.float32))
        probabilities =F.softmax(logits,dim=1)
        action_distribution = Categorical(probabilities)
        action = action_distribution.sample()
        log_prob = action_distribution.log_prob(action)
        entropy = action_distribution.entropy()
        return action, log_prob,entropy
    def one_gradient_step(self, env):
        # run trajectories until done
        log_probs = [[] for i in range(env.observation_space.shape[0])]
        returns = []
        x,_ = env.reset()
        rewards = [[] for i in range(env.observation_space.shape[0])]
        entropies = []
        episode_cum_reward = 0
        for i in range(200):
            a, log_prob ,entropy= self.sample_action_and_log_prob(x)
            y,r,d,trunc,infos = env.step(a.numpy())
            entropies.append(entropy)
            for i in range(env.observation_space.shape[0]): 
                log_probs[i].append(log_prob[i])
                rewards[i].append(r[i])
                episode_cum_reward += r
            x=y
        for i in range(len(rewards)):
            # compute returns-to-go
            new_returns = []
            G_t = 0
            for r in reversed(rewards[i]):
                G_t = r + self.gamma * G_t
                new_returns.append(G_t)
            new_returns = list(reversed(new_returns))
            returns.extend(new_returns)
            
        # make loss
        entropy_loss = -100*torch.cat(entropies).mean()
        returns = torch.tensor(returns)
        log_probs = torch.cat([torch.stack(u) for u in log_probs])
        loss = -(returns * log_probs).mean()+ entropy_loss.to(self.device)
        # gradient step
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return np.mean(episode_cum_reward)/env.observation_space.shape[0], entropy_loss.item(), loss.item()

    def train(self, env, nb_rollouts):
        avg_sum_rewards = []
        pbar = trange(nb_rollouts)
        for ep in pbar:
            avg,ent_loss ,loss= self.one_gradient_step(env)
            avg = descale(avg)
            pbar.set_postfix(avg=avg,ent_loss = ent_loss,loss= loss)
            avg_sum_rewards.append(avg)
        return avg_sum_rewards


In [5]:
"""import gymnasium as gym
import matplotlib.pyplot as plt
from env_hiv import *
env =gym.vector.AsyncVectorEnv([lambda : HIVPatient(domain_randomization = True) for i in range(30)])
config = {'gamma': 1,
          'learning_rate': 0.001
         }
from train import *
pi = ProjectAgent()
pi.load()

#ON VA FAIRE DU KAGGLE / DES AGENTS ENTRAINES AVEC OU SANS RANDOM, PLEINS
#pi = policyNetwork(env)

agent = reinforce_agent(config, pi)
returns = agent.train(env,50)
plt.plot(returns)"""

"import gymnasium as gym\nimport matplotlib.pyplot as plt\nfrom env_hiv import *\nenv =gym.vector.AsyncVectorEnv([lambda : HIVPatient(domain_randomization = True) for i in range(30)])\nconfig = {'gamma': 1,\n          'learning_rate': 0.001\n         }\nfrom train import *\npi = ProjectAgent()\npi.load()\n\n#ON VA FAIRE DU KAGGLE / DES AGENTS ENTRAINES AVEC OU SANS RANDOM, PLEINS\n#pi = policyNetwork(env)\n\nagent = reinforce_agent(config, pi)\nreturns = agent.train(env,50)\nplt.plot(returns)"

In [6]:
env = HIVPatient()
from gymnasium.wrappers import TransformReward
def rew(state,action, env) : 
    return-(
                env.Q * state[4]
                + env.R1 * action[0] ** 2
                + env.R2 * action[1] ** 2
                - env.S * state[5]
            )
low_r, high_r = rew(env.lower, [1,1],env),rew(env.upper, [1,1],env)
env = TransformReward(env, lambda r: (r-low_r)/(high_r-low_r))
scaledHIV = lambda domain_randomization : TransformReward(HIVPatient(domain_randomization), lambda r: (r-low_r)/(high_r-low_r))
def descale(r) :
    return r * (high_r - low_r) +low_r

In [7]:
import gymnasium as gym
import matplotlib.pyplot as plt
from env_hiv import *
import pickle
config = {'gamma': 1,
          'learning_rate': 0.001
         }
from train import *
for i in range(10) :
    env =gym.vector.AsyncVectorEnv([lambda : scaledHIV(domain_randomization = i%2) for j in range(30)])
    pi = ProjectAgent()
    pi.load()
    agent = reinforce_agent(config, pi)
    returns = agent.train(env,50)
    serialized= {"params" : pi.get_params()}
    with open('KAGGLETIER/saved'+str(i)+".pkl", 'wb') as f:  # open a text file
        pickle.dump(serialized, f) # serialize the list

 32%|███▏      | 16/50 [03:10<06:45, 11.92s/it, avg=2.73e+9, ent_loss=-2.67, loss=-2.41]


KeyboardInterrupt: 

In [ ]:
import os
class Aggregator : 
    def __init__(self) : 
        model_names = os.listdir("KAGGLETIER")
        self.models = []
        for name in model_names : 
            pi = ProjectAgent()
            with open('KAGGLETIER/'+name, 'rb') as f:  # open a text file
                d= pickle.load( f) # serialize the list
            pi.set_params(d['params'])
            self.models.append(pi)
    def act(self, obs):
        logits = [m(obs) for m in self.models]
        actions = []
        for l in logits : 
            probabilities = nn.Softmax(dim = 1)(l)
            action_distribution = Categorical(probabilities)
            actions.append(action_distribution.sample().item())
        counts = np.bincount(actions)
        return np.argmax(counts)


agg = Aggregator()
env = HIVPatient()



In [ ]:
agg = Aggregator()
env = HIVPatient(domain_randomization =True)
x,_ = env.reset()
for i in range(2):
    a = agg.act(x)
    x,_,_,_,_ = env.step(a)
    print([m(x) for m in agg.models][0])
    print(nn.Softmax(dim = 1)([m(x) for m in agg.models][0]))

tensor([[   0.0000,  526.3597, 1091.3484, 1042.7209]], grad_fn=<ReluBackward0>)
tensor([[0.0000e+00, 0.0000e+00, 1.0000e+00, 7.6098e-22]],
       grad_fn=<SoftmaxBackward0>)
tensor([[  0.0000,   0.0000, 438.5038,   0.0000]], grad_fn=<ReluBackward0>)
tensor([[0., 0., 1., 0.]], grad_fn=<SoftmaxBackward0>)


In [ ]:

from evaluate import evaluate_HIV, evaluate_HIV_population
import random
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)

seed_everything(seed=42)
# Initialization of the agent. Replace DummyAgent with your custom agent implementation.
agent = ProjectAgent()
agent.load()
# Keep the following lines to evaluate your agent unchanged.
score_agent: float = evaluate_HIV(agent=agent, nb_episode=1)
score_agent_dr: float = evaluate_HIV_population(agent=agent, nb_episode=15)
with open(file="score.txt", mode="w") as f:
    f.write(f"{score_agent}\n{score_agent_dr}")


KeyboardInterrupt: 

In [ ]:
a=9709249010.93793
b=2610906993.0028586
score_agent,score_agent_dr

(9709249010.93793, 2610906993.0028586)